# Spark Data Sources

This notebook shows how to use Spark Data Sources Interface API to read file formats:
 * Parquet
 * JSON
 * CSV
 * Avro
 * ORC
 * Image
 * Binary

A full list of DataSource methods is available [here](https://docs.databricks.com/spark/latest/data-sources/index.html#id1)

## Define paths for the various data sources

In [1]:
parquet_file = "./flights/summary-data/parquet/2010-summary.parquet"
json_file = "./flights/summary-data/json/*"
csv_file = "./flights/summary-data/csv/*"
orc_file = "./flights/summary-data/orc/*"
avro_file = "./flights/summary-data/avro/*"
schema = "DEST_COUNTRY_NAME STRING, ORIGIN_COUNTRY_NAME STRING, count INT"

## Parquet Data Source

In [4]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
spark = (
    SparkSession
    .builder
    .appName("04_chap")
    .getOrCreate()
    )
sc = spark.sparkContext

df = (spark
      .read
      .format("parquet")
      .option("path", parquet_file)
      .load())

Another way to read this same data using a variation of this API

In [5]:
df2 = spark.read.parquet(parquet_file)

In [6]:
df.show(10, False)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|United States    |Romania            |1    |
|United States    |Ireland            |264  |
|United States    |India              |69   |
|Egypt            |United States      |24   |
|Equatorial Guinea|United States      |1    |
|United States    |Singapore          |25   |
|United States    |Grenada            |54   |
|Costa Rica       |United States      |477  |
|Senegal          |United States      |29   |
|United States    |Marshall Islands   |44   |
+-----------------+-------------------+-----+
only showing top 10 rows



## Use SQL

This will create an _unmanaged_ temporary view

In [9]:
# %sql
spark.sql("""
CREATE OR REPLACE TEMPORARY VIEW us_delay_flights_tbl
    USING parquet
    OPTIONS (
      path "/databricks-datasets/definitive-guide/data/flight-data/parquet/2010-summary.parquet"
    )
""")

AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/databricks-datasets/definitive-guide/data/flight-data/parquet/2010-summary.parquet.

Use SQL to query the table

The outcome should be the same as one read into the DataFrame above

In [ ]:
spark.sql("SELECT * FROM us_delay_flights_tbl").show(10, truncate=False)

## JSON Data Source

In [ ]:
df = spark.read.format("json").option("path", json_file).load()

In [ ]:
df.show(10, truncate=False)

In [ ]:
df2 = spark.read.json(json_file)

In [ ]:
df2.show(10, False)

## Use SQL

This will create an _unmanaged_ temporary view

In [ ]:
# %sql
CREATE OR REPLACE TEMPORARY VIEW us_delay_flights_tbl
    USING json
    OPTIONS (
      path "/databricks-datasets/learning-spark-v2/flights/summary-data/json/*"
    )

Use SQL to query the table

The outcome should be the same as one read into the DataFrame above

In [ ]:
spark.sql("SELECT * FROM us_delay_flights_tbl").show(10, truncate=False)

## CSV Data Source

In [ ]:
df = (spark
      .read
	 .format("csv")
	 .option("header", "true")
	 .schema(schema)
	 .option("mode", "FAILFAST")  # exit if any errors
	 .option("nullValue", "")	  # replace any null data field with “”
	 .option("path", csv_file)
	 .load())


In [ ]:
df.show(10, truncate = False)

In [ ]:
(df.write.format("parquet")
  .mode("overwrite")
  .option("path", "/tmp/data/parquet/df_parquet")
  .option("compression", "snappy")
  .save())

In [ ]:
%fs ls /tmp/data/parquet/df_parquet

In [ ]:
df2 = (spark
       .read
       .option("header", "true")
       .option("mode", "FAILFAST")	 # exit if any errors
       .option("nullValue", "")
       .schema(schema)
       .csv(csv_file))

In [ ]:
df2.show(10, truncate=False)

## Use SQL

This will create an _unmanaged_ temporary view

In [ ]:
# %sql
CREATE OR REPLACE TEMPORARY VIEW us_delay_flights_tbl
    USING csv
    OPTIONS (
      path "/databricks-datasets/learning-spark-v2/flights/summary-data/csv/*",
      header "true",
      inferSchema "true",
      mode "FAILFAST"
    )

Use SQL to query the table

The outcome should be the same as one read into the DataFrame above

In [ ]:
spark.sql("SELECT * FROM us_delay_flights_tbl").show(10, truncate=False)

## ORC Data Source

In [ ]:
df = (spark.read
      .format("orc")
      .option("path", orc_file)
      .load())

In [ ]:
df.show(10, truncate=False)

## Use SQL

This will create an _unmanaged_ temporary view

In [ ]:
# %sql
CREATE OR REPLACE TEMPORARY VIEW us_delay_flights_tbl
    USING orc
    OPTIONS (
      path "/databricks-datasets/learning-spark-v2/flights/summary-data/orc/*"
    )

Use SQL to query the table

The outcome should be the same as one read into the DataFrame above

In [ ]:
spark.sql("SELECT * FROM us_delay_flights_tbl").show(10, truncate=False)

## Avro Data Source

In [ ]:
df = (spark.read
      .format("avro")
      .option("path", avro_file)
      .load())

In [ ]:
df.show(10, truncate=False)

## Use SQL

This will create an _unmanaged_ temporary view

In [ ]:
# %sql
CREATE OR REPLACE TEMPORARY VIEW us_delay_flights_tbl
    USING avro
    OPTIONS (
      path "/databricks-datasets/learning-spark-v2/flights/summary-data/avro/*"
    )

Use SQL to query the table

The outcome should be the same as the one read into the DataFrame above

In [ ]:
spark.sql("SELECT * FROM us_delay_flights_tbl").show(10, truncate=False)

## Image

In [ ]:
from pyspark.ml import image

image_dir = "/databricks-datasets/cctvVideos/train_images/"
images_df = spark.read.format("image").load(image_dir)
images_df.printSchema()

images_df.select("image.height", "image.width", "image.nChannels", "image.mode", "label").show(5, truncate=False)

## Binary

In [ ]:
path = "/databricks-datasets/learning-spark-v2/cctvVideos/train_images/"
binary_files_df = (spark
                   .read
                   .format("binaryFile")
                   .option("pathGlobFilter", "*.jpg")
                   .load(path))
binary_files_df.show(5)

To ignore any partitioning data discovery in a directory, you can set the `recursiveFileLookup` to `true`.

In [ ]:
binary_files_df = (spark
                   .read
                   .format("binaryFile")
                   .option("pathGlobFilter", "*.jpg")
                   .option("recursiveFileLookup", "true")
                   .load(path))
binary_files_df.show(5)